# **Context**
**What is Phishing?**

Phishing is a type of cybercrime in which a target or targets are contacted by email, telephone or text message by someone posing as a legitimate institution to lure individuals into providing sensitive data such as personally identifiable information, banking and credit card details, and passwords.

The information is then used to access important accounts of the victims and it mostly results in identity theft and financial loss.

Phishing is one of the most popular cybercrimes among attackers as it is easier to trick someone into clicking a malicious link which seems legitimate than trying to break through a computer’s defence systems.

**Problem Statement**
Need to develop a machine learning detector that would automatically detect phishing attacks.

**Evaluation Criteria**
Accuracy Score.

**Machine Learning Problem Type**
Classification



In [741]:
#!pip install catboost
# Importing Basic Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [742]:
# Importing the Data
phishing_data  = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/phishing_data/Training_set_label.csv")#,index_col='url'
test_data = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/phishing_data/Testing_set_label.csv')#,index_col='url'

In [743]:
# Exploring the Train Data
phishing_data.sample(10)

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,nb_underscore,nb_tilde,nb_percent,nb_slash,nb_star,nb_colon,nb_comma,nb_semicolumn,nb_dollar,nb_space,nb_www,nb_com,nb_dslash,http_in_path,https_token,ratio_digits_url,ratio_digits_host,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,nb_subdomains,prefix_suffix,random_domain,shortening_service,path_extension,nb_redirection,nb_external_redirection,length_words_raw,char_repeat,shortest_words_raw,shortest_word_host,shortest_word_path,longest_words_raw,longest_word_host,longest_word_path,avg_words_raw,avg_word_host,avg_word_path,phish_hints,domain_in_brand,brand_in_subdomain,brand_in_path,suspecious_tld,statistical_report,nb_hyperlinks,ratio_intHyperlinks,ratio_extHyperlinks,ratio_nullHyperlinks,nb_extCSS,ratio_intRedirection,ratio_extRedirection,ratio_intErrors,ratio_extErrors,login_form,external_favicon,links_in_tags,submit_email,ratio_intMedia,ratio_extMedia,sfh,iframe,popup_window,safe_anchor,onmouseover,right_clic,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
3602,http://dict.hinkhoj.com/latency-meaning-in-hin...,54,16,0,3,3,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,0,0,0,0,1,0.000000,0.000000,0,0,0,0,0,3,0,0,0,0,1,0,7,0,2,4,2,7,7,7,5.285714,5.500000,5.200000,0,0,0,0,0,0,64,0.640625,0.359375,0,1,0,0.173913,0,0.000000,0,1,53.846154,0,84.210526,15.789474,0,0,0,55.000000,0,0,0,1,one,0,1972,4968,8541,0,0,4,legitimate
1403,http://DateAndTime.info/city.php?id=3573890,43,16,1,2,0,0,1,0,0,1,0,0,0,3,0,1,0,0,0,0,0,0,0,0,1,0.162791,0.000000,0,0,0,0,0,2,0,0,0,0,1,0,5,0,2,11,2,11,11,7,5.400000,11.000000,4.000000,0,0,0,0,0,0,117,0.948718,0.051282,0,0,0,0.500000,0,0.000000,0,0,62.500000,0,100.000000,0.000000,0,0,0,76.923077,0,0,0,1,zero,1,72,-1,101802,0,0,5,legitimate
1196,http://www.oliopizzeria.com/,28,20,0,2,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,1,0,0,0,1,0.000000,0.000000,0,0,0,0,0,2,0,0,0,0,1,0,2,4,3,3,0,12,12,0,7.500000,7.500000,0.000000,0,0,0,0,0,0,42,0.547619,0.452381,0,2,0,0.052632,0,0.000000,0,1,60.000000,0,0.000000,0.000000,0,0,0,45.833333,0,0,0,1,one,0,1692,3787,4702582,0,0,3,legitimate
7830,http://www.wellbeingresort-ci.com/images/webma...,68,26,0,3,1,0,0,0,0,0,0,0,0,6,0,1,0,0,0,0,1,0,0,0,1,0.000000,0.000000,0,0,0,0,0,3,1,0,0,0,1,0,8,4,2,2,3,15,15,7,6.250000,6.666667,6.000000,1,0,0,0,0,0,6,1.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,100.000000,0,100.000000,0.000000,0,0,0,0.000000,0,0,0,1,one,0,23,1072,0,0,1,0,phishing
589,http://www.megavideosdesexo.com/,32,24,0,2,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,1,0,0,0,1,0.000000,0.000000,0,0,0,0,0,2,0,0,0,0,0,0,2,3,3,3,0,16,16,0,9.500000,9.500000,0.000000,0,0,0,0,0,0,49,0.428571,0.571429,0,0,0,0.285714,0,0.107143,0,0,75.000000,0,0.000000,0.000000,0,1,0,20.588235,0,0,0,1,one,0,228,2329,3474559,0,0,1,legitimate
5751,http://plataforma-segwra-bnonline1.com/BNWeb/l...,51,31,0,1,2,0,0,0,0,0,0,0,0,4,0,1,0,0,0,0,0,0,0,0,1,0.019608,0.032258,0,0,0,0,0,1,0,0,0,0,1,0,5,0,5,6,5,10,10,6,7.200000,8.333333,5.500000,0,0,0,0,0,0,34,1.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,100.000000,0,100.000000,0.000000,0,0,0,100.000000,0,0,0,1,One,0,316,-1,0,0,1,0,phishing
11121,https://www.secureupdate.appleld.com.duilawyer...,124,46,1,5,0,0,1,2,0,3,2,0,0,5,0,1,0,0,0,0,1,1,0,0,0,0.153226,0.000000,0,0,0,1,0,3,0,0,0,0,0,0,13,4,2,3,2,17,13,17,7.461538,7.600000,7.375000,0,0,0,0,0,0,51,1.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,100.000000,0,100.000000,0.000000,0,0,0,100.000000,0,0,0,1,one,0,25,3992,5707171,0,1,0,phishing
9626,https://www.arrt.org/docs/default-source/disci...,118,12,0,3,5,0,1,0,0,1,0,0,0,7,0,1,0,0,0,0,1,1,0,0,0,0.016949,0.000000,0,0,0,0,0,3,1,0,0,0,1,0,15,4,2,3,2,10,4,10,6.133333,3.500000,6.538462,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000,0.000000,0,0,0,0.000000,0,0,1,1,One,0,198,8569,492031,0,0,4,legitimate
8100,http://security.paypal.com.userid.874585.compr...

In [744]:
# train and test data shape
phishing_data_row,phishing_data_column=phishing_data.shape
test_data_row,test_data_column=test_data.shape
print("Number of Rows in Train data: {}, Number of Rows in Test data: {}".format(phishing_data_row,test_data_row))
print("Number of columns in Train data: {}, Number of columns in Test data: {}".format(phishing_data_column,test_data_column))

Number of Rows in Train data: 11481, Number of Rows in Test data: 3429
Number of columns in Train data: 89, Number of columns in Test data: 88


In [745]:
# merging train and test data for preprocesiing the data
phishing_data['train_data']=1
test_data['train_data']=0

merged_data=phishing_data.append(test_data)
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14910 entries, 0 to 3428
Data columns (total 90 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   url                         14910 non-null  object 
 1   length_url                  14910 non-null  int64  
 2   length_hostname             14910 non-null  int64  
 3   ip                          14910 non-null  object 
 4   nb_dots                     14910 non-null  int64  
 5   nb_hyphens                  14910 non-null  object 
 6   nb_at                       14910 non-null  int64  
 7   nb_qm                       14910 non-null  int64  
 8   nb_and                      14910 non-null  int64  
 9   nb_or                       14910 non-null  int64  
 10  nb_eq                       14910 non-null  int64  
 11  nb_underscore               14910 non-null  int64  
 12  nb_tilde                    14910 non-null  int64  
 13  nb_percent                  1491

**Observation**
*   No Null Values in the dataset
*   'ip', 'nb_hyphens' columns of 'object' dtype . so we need check the unique values of those columns, if any string values is present.
*   'domain_with_copyright','status' columns need to label encoded.

In [746]:
print("Unique Values in 'ip' column are ",merged_data['ip'].unique())
print("Unique Values in 'nb_hyphens' column are ",merged_data['nb_hyphens'].unique())
print("Unique Values in 'domain_with_copyright' column are ",merged_data['domain_with_copyright'].unique())
print("Unique Values in 'status' column are ",merged_data['status'].unique())

Unique Values in 'ip' column are  ['zero' '0' '1' 'one']
Unique Values in 'nb_hyphens' column are  ['zero' '0' '1' '2' '5' '7' '4' '3' 'one' '8' '9' '6' '17' '11' '14' '24'
 '10' '13' '15' '19' '18' '16' '20' '26' '12' '32' '23' '43' '28']
Unique Values in 'domain_with_copyright' column are  ['one' 'zero' 'One' 'Zero']
Unique Values in 'status' column are  ['phishing' 'legitimate' nan]


**Observation**
*   'ip', 'nb_hyphens', 'domain_with_copyright' columns have few string values. We need to replace the string with corresponding int values.
* The target column 'status' need to be labelencoded.

In [747]:
# replacing the string values into int values
enco_ip_dic={'zero':0,'one':1,'0':0,'1':1}
enco_hyp_dic={'zero':0, '0':0,'1':1, '2':2, '5':5, '7':7, '4':4, '3':3, 'one':1, '8':8, '9':9, '6':6, '17':17, '11':11, '14':14, '24':24,'10':10, '13':13, '15':15, '19':19, '18':18, '16':16, '20':20, '26':26, '12':12, '32':32, '23':23, '43':43, '28':28}
enco_dom_dic={'one':1,'zero':0,'One':1,'Zero':0}
enco_sta_dic={'phishing':1,'legitimate':0}

merged_data['ip']=merged_data['ip'].map(enco_ip_dic)
merged_data['nb_hyphens']=merged_data['nb_hyphens'].map(enco_hyp_dic)
merged_data['domain_with_copyright']=merged_data['domain_with_copyright'].map(enco_dom_dic)
merged_data['status']=merged_data['status'].map(enco_sta_dic)

In [748]:
# Summary statistics
merged_data.describe()

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,nb_underscore,nb_tilde,nb_percent,nb_slash,nb_star,nb_colon,nb_comma,nb_semicolumn,nb_dollar,nb_space,nb_www,nb_com,nb_dslash,http_in_path,https_token,ratio_digits_url,ratio_digits_host,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,nb_subdomains,prefix_suffix,random_domain,shortening_service,path_extension,nb_redirection,nb_external_redirection,length_words_raw,char_repeat,shortest_words_raw,shortest_word_host,shortest_word_path,longest_words_raw,longest_word_host,longest_word_path,avg_words_raw,avg_word_host,avg_word_path,phish_hints,domain_in_brand,brand_in_subdomain,brand_in_path,suspecious_tld,statistical_report,nb_hyperlinks,ratio_intHyperlinks,ratio_extHyperlinks,ratio_nullHyperlinks,nb_extCSS,ratio_intRedirection,ratio_extRedirection,ratio_intErrors,ratio_extErrors,login_form,external_favicon,links_in_tags,submit_email,ratio_intMedia,ratio_extMedia,sfh,iframe,popup_window,safe_anchor,onmouseover,right_clic,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status,train_data
count,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.0,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.00000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.0,14910.000000,14910.0,14910.000000,14910.0,14910.000000,14910.000000,14910.000000,14910.000000,14910.0,14910.000000,14910.000000,14910.0,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,14910.000000,1.491000e+04,14910.000000,14910.000000,14910.000000,11481.000000,14910.000000
mean,61.175922,21.106237,0.149363,2.486452,0.991214,0.022602,0.141247,0.163917,0.0,0.294098,0.329108,0.006774,0.121730,4.299531,0.000604,1.027767,0.004494,0.062710,0.001744,0.037827,0.448826,0.128169,0.006237,0.016432,0.611871,0.053092,0.024955,0.000469,0.00228,0.065661,0.050302,0.021663,2.235144,0.202482,0.082562,0.120389,0.000134,0.499866,0.003152,6.251844,2.912542,3.126693,5.038565,2.409390,15.348759,10.477197,10.523742,7.251472,7.688274,5.098533,0.329309,0.103555,0.004427,0.004762,0.018511,0.060630,86.478203,0.602351,0.275315,0.0,0.782964,0.0,0.157748,0.0,0.062778,0.063581,0.440376,51.984579,0.0,42.647182,23.067103,0.0,0.001274,0.005969,37.124090,0.001207,0.001408,0.124950,0.775587,0.440174,0.073843,492.864856,4059.293092,8.580428e+05,0.019450,0.535077,3.183769,0.500044,0.770020
std,55.271915,10.742024,0.356458,1.391955,2.052073,0.156120,0.364842,0.833088,0.0,1.004630,1.126575,0.082028,1.392998,1.897542,0.024562,0.241636,0.108866,0.593544,0.072310,0.415885,0.502089,0.380107,0.078733,0.171200,0.487340,0.089240,0.093227,0.021663,0.04770,0.247696,0.218574,0.145587,0.637314,0.401863,0.275228,0.325427,0.011581,0.692296,0.056058,5.601911,4.557529,2.211981,3.971365,3.004603,21.864841,4.962572,22.848818,4.122077,3.599656,7.088885,0.847049,0.304692,0.066387,0.068844,0.134795,0.333799,163.978093,0.376858,0.319393,0.0,2.689759,0.0,0.265159,0.0,0.156450,0.244014,0.496449,41.546250,0.0,46.248809,38.309141,0.0,0.035676,0.077032,39.096864,0.034726,0.037504,0.330673,0.417209,0.496425,0.261524,826.130806,3110.015543,1.995794e+06,0.138105,0.498785,2.534337,0.500022,0.420834
min,12.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,

In [749]:
# taking log transformation on few columns
merged_data['length_hostname']=np.log(merged_data['length_hostname'])
merged_data['length_words_raw']=np.log(merged_data['length_words_raw'])
merged_data['longest_words_raw']=np.log(merged_data['longest_words_raw'])
merged_data['ratio_digits_host']=np.log1p(merged_data['ratio_digits_host'])
merged_data['shortest_word_host']=np.log1p(merged_data['shortest_word_host'])
merged_data['shortest_word_path']=np.log1p(merged_data['shortest_word_path'])
merged_data['ratio_extErrors']=np.log1p(merged_data['ratio_extErrors'])

In [750]:
# again splitting the data to train and test
train=merged_data.iloc[:11481,:]
test=merged_data.iloc[11481:,:]

In [751]:
# exploring the top 10 rows of train data
train.head(10)

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,nb_underscore,nb_tilde,nb_percent,nb_slash,nb_star,nb_colon,nb_comma,nb_semicolumn,nb_dollar,nb_space,nb_www,nb_com,nb_dslash,http_in_path,https_token,ratio_digits_url,ratio_digits_host,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,nb_subdomains,prefix_suffix,random_domain,shortening_service,path_extension,nb_redirection,nb_external_redirection,length_words_raw,char_repeat,shortest_words_raw,shortest_word_host,shortest_word_path,longest_words_raw,longest_word_host,longest_word_path,avg_words_raw,avg_word_host,avg_word_path,phish_hints,domain_in_brand,brand_in_subdomain,brand_in_path,suspecious_tld,statistical_report,nb_hyperlinks,ratio_intHyperlinks,ratio_extHyperlinks,ratio_nullHyperlinks,nb_extCSS,ratio_intRedirection,ratio_extRedirection,ratio_intErrors,ratio_extErrors,login_form,external_favicon,links_in_tags,submit_email,ratio_intMedia,ratio_extMedia,sfh,iframe,popup_window,safe_anchor,onmouseover,right_clic,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status,train_data
0,http://www.progarchives.com/album.asp?id=61737,46,2.995732,0,3,0,0,1,0,0,1,0,0,0,3,0,1,0,0,0,0,1,0,0,0,1,0.108696,0.000000,0,0,0,0,0,3,0,0,0,0,0,0,1.791759,3,2,1.386294,1.098612,2.484907,12,5,5.000000,7.5,3.750000,0,0,0,0,0,0,143,0.930070,0.069930,0,1,0,0.000000,0,0.000000,0,1,73.913043,0,100.000000,0.000000,0,0,0,77.777778,0,0,0,1,1,0,627,6678,78526,0,0,5,1.0,1
1,http://signin.eday.co.uk.ws.edayisapi.dllsign....,128,4.787492,0,10,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,0,0,0,0,1,0.054688,0.056695,0,0,0,0,0,3,0,0,0,0,0,0,2.302585,6,2,1.098612,0.000000,3.555348,35,0,10.700000,10.7,0.000000,2,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000,0.000000,0,0,0,0.000000,0,0,1,1,0,0,300,65,0,0,1,0,1.0,1
2,http://www.avevaconstruction.com/blesstool/ima...,52,3.218876,0,3,0,0,0,0,0,0,0,0,0,4,0,1,0,0,0,0,1,0,0,0,1,0.000000,0.000000,0,0,0,0,0,3,0,0,0,0,1,0,1.609438,5,3,1.386294,1.386294,2.833213,17,9,7.400000,10.0,5.666667,0,0,0,0,0,0,3,1.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,100.000000,0,0.000000,0.000000,0,0,0,0.000000,0,0,0,1,0,0,119,1707,0,0,1,0,1.0,1
3,http://www.jp519.com/,21,2.564949,0,2,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,1,0,0,0,1,0.142857,0.207639,0,0,0,0,0,2,0,1,0,0,0,0,0.693147,3,3,1.386294,0.000000,1.609438,5,0,4.000000,4.0,0.000000,0,0,0,0,0,0,404,0.962871,0.037129,0,0,0,0.133333,0,0.000000,0,0,100.000000,0,92.307692,7.692308,0,0,0,82.539683,0,0,0,1,1,0,130,1331,0,0,0,0,0.0,1
4,https://www.velocidrone.com/,28,2.944439,0,2,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,1,0,0,0,0,0.000000,0.000000,0,0,0,0,0,2,0,0,0,0,0,0,0.693147,3,3,1.386294,0.000000,2.397895,11,0,7.000000,7.0,0.000000,0,0,0,0,0,0,57,0.684211,0.315789,0,3,0,0.000000,0,0.000000,0,1,55.555556,0,50.000000,50.000000,0,0,0,81.081081,0,0,0,0,0,0,164,1662,312044,0,0,4,0.0,1
5,https://support-appleld.com.secureupdate.duila...,128,3.912023,1,4,1,0,1,2,0,3,2,0,0,5,0,1,0,0,0,0,0,1,0,0,0,0.117188,0.000000,0,0,0,1,0,3,1,0,0,0,0,0,2.564949,4,2,1.386294,1.098612,2.833213,13,17,7.769231,8.4,7.375000,0,0,0,0,0,0,51,1.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,100.000000,0,100.000000,0.000000,0,0,0,100.000000,0,0,0,1,1,0,25,3993,5707171,0,1,0,1.0,1
6,https://www.authpro.com/auth/ubabankng/?action...,50,2.708050,0,2,0,0,1,0,0,1,0,0,0,5,0,1,0,0,0,0,1,0,0,0,0,0.000000,0.000000,0,0,0,0,0,2,0,0,0,0,0,0,1.791759,3,3,1.386294,1.386294,2.197225,7,9,5.333333,5.0,5.500000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000,0.000000,0,0,0,0.000000,0,0,0,1,0,0,705,7330,154708,0,0,4,1.0,1
7,http://littlee.com.au/alibaba/login.alibaba.co...,51,2.639057,0,5,0,0,0,0,0,0,0,0,0,4,0,1,0,0,0,0,0,1,0,0,1,0.000000,0.000000,0,0,0,0,0,3,0,0,0,0,1,0,1.791759,2,3,2.079442,1.386294,1.945910,7,7,5.333333,7.0,5.000000,3,0,0,1,0,0,4,1.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,

In [752]:
# target Value distibution
train['status'].value_counts()

1.0    5741
0.0    5740
Name: status, dtype: int64

Observation

Dataset is Balanced.

In [753]:
num_col=train.select_dtypes(exclude=['object']).columns
col_cat=[]
col_2=[]
for col in num_col: 
  x=train[col].nunique()
  print('Number of unique values in {} : {}'.format(col,x))
  if x==1:
    col_cat.append(col)
  if x==2:
    if col!='status':
      col_2.append(col)

print(col_cat)#column with 1 unique values
print(col_2)#column with 2 unique values

Number of unique values in length_url : 291
Number of unique values in length_hostname : 78
Number of unique values in ip : 2
Number of unique values in nb_dots : 19
Number of unique values in nb_hyphens : 25
Number of unique values in nb_at : 4
Number of unique values in nb_qm : 4
Number of unique values in nb_and : 14
Number of unique values in nb_or : 1
Number of unique values in nb_eq : 15
Number of unique values in nb_underscore : 17
Number of unique values in nb_tilde : 2
Number of unique values in nb_percent : 19
Number of unique values in nb_slash : 21
Number of unique values in nb_star : 2
Number of unique values in nb_colon : 6
Number of unique values in nb_comma : 5
Number of unique values in nb_semicolumn : 15
Number of unique values in nb_dollar : 3
Number of unique values in nb_space : 9
Number of unique values in nb_www : 3
Number of unique values in nb_com : 6
Number of unique values in nb_dslash : 2
Number of unique values in http_in_path : 5
Number of unique values in

* 'nb_or', 'ratio_nullHyperlinks', 'ratio_intRedirection', 'ratio_intErrors', 'submit_email', 'sfh', 'train_data'. These columns have only 1 unique value. so we drop these columns.

In [754]:
#drop the column with 1 unique value.
train=train.drop(col_cat,axis=1)
test=test.drop(col_cat,axis=1)

In [755]:
# import the libraries
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.metrics import classification_report,confusion_matrix
import xgboost as xgb

In [756]:
#train['url_security']=train['url'].str.split(':').str.get(0).str.len()

In [757]:
X=train.drop(['url','status'],axis=1)
y=train[['status']]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=3)

"""param={"max_depth":[3,5,7], 'learning_rate':[0.1], 'n_estimators':[300,500,],}
estimator=xgb.XGBClassifier(objective='binary:logistic')
model=RandomizedSearchCV(estimator,param_distributions=param,scoring='accuracy',n_jobs=-1,cv=10,verbose=1,return_train_score=True,)"""

model=xgb.XGBClassifier(objective='binary:logistic',learning_rate= 0.1, max_depth=7, n_estimators=300)
model.fit(X_train,y_train)
y_predict=model.predict(X_test)
print(confusion_matrix(y_test,y_predict))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[1143   22]
 [  19 1113]]


In [758]:
# extracting the important columns
important_col=[]
for x in zip(np.round(model.feature_importances_,2),X.columns):
  if x[0]>0:
    important_col.append(x[1])
  print(x)
print(important_col)

(0.0, 'length_url')
(0.01, 'length_hostname')
(0.01, 'ip')
(0.01, 'nb_dots')
(0.01, 'nb_hyphens')
(0.0, 'nb_at')
(0.02, 'nb_qm')
(0.01, 'nb_and')
(0.01, 'nb_eq')
(0.01, 'nb_underscore')
(0.0, 'nb_tilde')
(0.01, 'nb_percent')
(0.01, 'nb_slash')
(0.0, 'nb_star')
(0.0, 'nb_colon')
(0.0, 'nb_comma')
(0.0, 'nb_semicolumn')
(0.0, 'nb_dollar')
(0.01, 'nb_space')
(0.03, 'nb_www')
(0.0, 'nb_com')
(0.0, 'nb_dslash')
(0.03, 'http_in_path')
(0.0, 'https_token')
(0.0, 'ratio_digits_url')
(0.02, 'ratio_digits_host')
(0.0, 'punycode')
(0.0, 'port')
(0.0, 'tld_in_path')
(0.0, 'tld_in_subdomain')
(0.0, 'abnormal_subdomain')
(0.0, 'nb_subdomains')
(0.0, 'prefix_suffix')
(0.0, 'random_domain')
(0.01, 'shortening_service')
(0.0, 'path_extension')
(0.0, 'nb_redirection')
(0.0, 'nb_external_redirection')
(0.01, 'length_words_raw')
(0.01, 'char_repeat')
(0.0, 'shortest_words_raw')
(0.01, 'shortest_word_host')
(0.01, 'shortest_word_path')
(0.01, 'longest_words_raw')
(0.0, 'longest_word_host')
(0.01, 'longest_

In [763]:
#test prediction
test=test.drop(['url','status'],axis=1)
test['prediction']=model.predict(test)
test['prediction']=test['prediction'].map({1:'phishing',0:'legitimate'})
test['prediction'].to_csv('output.csv',index=False)